# Estimate Cam06 calibration.

We forgot to take calibrate Cam06 after it was upgraded in April, and didn't realize this unit after the BTF shut down in August 2022. Here, we estimate the mm/pixel calibration from a separate slit-slit measurmennt.

In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import json
from pprint import pprint
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from plotly import graph_objects as go
import proplot as pplt
from ipywidgets import interactive
from ipywidgets import widgets

sys.path.append('../..')
from tools.energyVS06 import EnergyCalculate
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup

In [ ]:
folder = '_saved/2022-07-15-VS06/'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
print('info')
pprint(info)

In [ ]:
datadir = info['datadir']
filename = info['filename']
datadir = info['datadir']
file = h5py.File(join(datadir, 'preproc-' + filename + '.h5'), 'r')
data_sc = file['/scalardata']
data_wf = file['/wfdata']
data_im = file['/imagedata']

print('Attributes:')
print()
for data in [data_sc, data_wf, data_im]:
    print(data.name)
    for item in data.dtype.fields.items():
        print(item)
    print()

In [ ]:
variables = info['variables']
keys = list(variables)
nsteps = np.array([variables[key]['steps'] for key in keys])

acts = info['acts']
print(acts)
points = np.vstack([data_sc[act] for act in acts]).T
points = points[:, ::-1]  # y, x2, x --> x, x2, y
nsteps = nsteps[::-1]

cam = info['cam']
print(f"cam = '{cam}'")
if cam.lower() not in ['cam06', 'cam34']:
    raise ValueError(f"Unknown camera name '{cam}'.")

## Convert to beam frame coordinates

In [ ]:
## y_slit is inserted from above, always opposite y_beam.
points[:, 2] = -points[:, 2]

## Screen coordinates (x3, y3) are always opposite beam (x, y).
image_shape = info['image_shape']
x3grid = -np.arange(image_shape[1]) * info['image_pix2mm_x'] 
y3grid = -np.arange(image_shape[0]) * info['image_pix2mm_y']

# VT04/VT06 are same sign as x_beam; VT34a and VT34b are opposite.
if cam.lower() == 'cam34':
    points[:, :2] = -points[:, :2]

## Setup interpolation grids

Build the transfer matrices between the slits and the screen. 

In [ ]:
# Eventually switch to saving metadata dict as pickled file in Step 0, then loading here.
_file = h5py.File(join(datadir, filename + '.h5'), 'r')
if 'config' in _file:
    config = _file['config']
    metadata = dict()
    for name in config['metadata'].dtype.names:
        metadata[name] = config['metadata'][name]
else:
    # Older measurement; metadata is in json file.
    metadata = json.load(open(join(datadir, filename + '-metadata.json'), 'r'))
    _metadata = dict()
    for _dict in metadata.values():
        for key, value in _dict.items():
            _metadata[key] = value
    metadata = _metadata
pprint(metadata)
_file.close()

In [ ]:
dipole_current = 0.0  # deviation of dipole current from nominal
l = 0.129  # dipole face to screen (assume same for first/last dipole-screen)
if cam.lower() == 'cam06':
    GL05 = 0.0  # QH05 integrated field strength (1 [A] = 0.0778 [Tm])
    GL06 = 0.0  # QH06 integrated field strength (1 [A] = 0.0778 [Tm])
    l1 = 0.280  # slit1 to QH05 center
    l2 = 0.210  # QH05 center to QV06 center
    l3 = 0.457  # QV06 center to slit2
    L2 = 0.599  # slit2 to dipole face    
    rho_sign = +1.0  # dipole bend radius sign
    if GL05 == 0.0 and metadata['BTF_MEBT_Mag:PS_QH05:I_Set'] != 0.0:
        print('Warning: QH05 is turned on according to metadata.')
    if GL05 != 0.0 and metadata['BTF_MEBT_Mag:PS_QH05:I_Set'] == 0.0:
        print('Warning: QH05 is turned off according to metadata.')
    if GL06 == 0.0 and metadata['BTF_MEBT_Mag:PS_QV06:I_Set'] != 0.0:
        print('Warning: QH06 is turned on according to metadata.')
    if GL06 != 0.0 and metadata['BTF_MEBT_Mag:PS_QV06:I_Set'] == 0.0:
        print('Warning: QH06 is turned off according to metadata.')
elif cam.lower() == 'cam34':
    GL05 = 0.0  # QH05 integrated field strength
    GL06 = 0.0  # QH06 integrated field strength
    l1 = 0.000  # slit1 to QH05 center
    l2 = 0.000  # QH05 center to QV06 center
    l3 = 0.774  # QV06 center to slit2
    L2 = 0.311  # slit2 to dipole face
    # Weird... I can only get the right answer for energy if I *do not* flip rho,
    # x1, x2, and x3. I then flip x and xp at the very end.
    rho_sign = +1.0  # dipole bend radius sign
    x3grid = -x3grid
    points[:, :2] = -points[:, :2]
LL = l1 + l2 + l3 + L2  # distance from emittance plane to dipole entrance
ecalc = EnergyCalculate(l1=l1, l2=l2, l3=l3, L2=L2, l=l, rho_sign=rho_sign)
Mslit = ecalc.getM1(GL05=GL05, GL06=GL06)  # slit-slit
Mscreen = ecalc.getM(GL05=GL05, GL06=GL06)  # slit-screen

Convert to x'.

In [ ]:
points[:, 1] = 1e3 * ecalc.calculate_xp(points[:, 0] * 1e-3, points[:, 1] * 1e-3, Mslit) 

Center points at zero.

In [ ]:
points -= np.mean(points, axis=0)

## Estimate pix2mm calibration from y-y'

In [ ]:
mins = np.min(points, axis=0)
maxs = np.max(points, axis=0)
scales = [1.1, 1.6, 1.1]
ns = np.multiply(scales, nsteps + 1).astype(int)
xgrid, xpgrid, ygrid = [np.linspace(umin, umax, n) for (umin, umax, n) in zip(mins, maxs, ns)]

iterations = data_sc['iteration']
iteration_nums = np.unique(iterations)
n_iterations = len(iteration_nums)
kws = dict(kind='linear', copy=True, bounds_error=False, fill_value=0.0, assume_sorted=False)

f_yy3 = np.zeros((len(ygrid), len(y3grid)))
for iteration in tqdm(iteration_nums):
    idx, = np.where(iterations == iteration)
    _points = points[idx, 2]
    _values = data_im[idx, cam + '_Image'].reshape((len(idx), len(y3grid), len(x3grid))).sum(axis=-1)
    _, uind = np.unique(_points, return_index=True)
    fint = interpolate.interp1d(_points[uind], _values[uind], axis=0, **kws)
    f_yy3 += fint(ygrid)
f_yy3 = f_yy3 / n_iterations

In [ ]:
fig, ax = pplt.subplots()
mplt.plot_image(f_yy3, x=ygrid, y=y3grid, ax=ax, colorbar=True)
ax.format(xlabel="y [mm]", ylabel="y3 [mm]", title='Original pix2mm')

In [ ]:
old_pix2mm = info['cam_pix2mm_x']  # zoom=0.33, raw image
new_pix2mm = 0.08  # zoom=0.33, raw image
ratio = new_pix2mm / old_pix2mm

print('old pix2mm:', old_pix2mm)
print('estimated new pix2mm:', new_pix2mm)
print('ratio:', ratio)

In [ ]:
fac = ratio  # scale assumed (wrong) pix2mm by this factor
print('Old pix2mm =', info['cam_pix2mm_x'] / 3.0)
print('New pix2mm =', fac * info['cam_pix2mm_x'] / 3.0)
_x3grid = x3grid * fac
_y3grid = y3grid * fac

YP = np.zeros((len(ygrid), len(_y3grid)))
for k, y in enumerate(ygrid):
    YP[k] = 1e3 * ecalc.calculate_yp(1e-3 * y, 1e-3 * _y3grid, Mscreen)
ypgrid = np.linspace(np.min(YP), np.max(YP), int(1.1 * len(_y3grid)))

W = np.zeros((len(xgrid), len(xpgrid), len(_x3grid)))
for i, x in enumerate(xgrid):
    for j, xp in enumerate(xpgrid):
        W[i, j] = ecalc.calculate_dE_screen(1e-3 * _x3grid, dipole_current, 1e-3 * x, 1e-3 * xp, Mscreen)
wgrid = np.linspace(np.min(W), np.max(W), int(1.1 * len(_x3grid)))

# Interpolate y3 -> y'.
f_yyp = np.zeros((len(ygrid), len(ypgrid)))
for i in range(len(ygrid)):
    _points = YP[i, :]
    _values = f_yy3[i, :]
    fint = interpolate.interp1d(_points, _values, axis=0, **kws)
    f_yyp[i, :] = fint(ypgrid)
        
fig, ax = pplt.subplots()
_ygrid = ygrid - np.average(ygrid, weights=np.sum(f_yyp, axis=1))
_ypgrid = ypgrid - np.average(ypgrid, weights=np.sum(f_yyp, axis=0))
mplt.plot_image(f_yyp / np.max(f_yyp), x=_ygrid, y=_ypgrid, ax=ax, colorbar=True, 
                # discrete=True,
                norm='log', vmin=0.001, vmax=1.0,
                thresh=1e-4, thresh_type='frac',
                contour=True, 
                contour_kws=dict(levels=[0.001, 0.01, 0.1, 0.5], labels=True),
                )
ax.format(xlabel="y [mm]", ylabel="yp [mrad]", title='New pix2mm')
ax.format(xlim=(-12.0, 12.0), ylim=(-7.5, 7.5))
plt.show()

from tools import analysis as ba
Sigma, mu = ba.dist_cov(f_yyp, (ygrid, ypgrid), disp=False)
print(Sigma)
ba.beam_stats(Sigma)

Compute rms energy spread as well.

In [ ]:
_x3grid = fac * x3grid
XXP = []
pws = []
for iteration in tqdm(iteration_nums):
    idx, = np.where(iterations == iteration)
    x, xp = np.mean(points[idx, :2], axis=0)
    _points = ecalc.calculate_dE_screen(1e-3 * _x3grid, dipole_current, 1e-3 * x, 1e-3 * xp, Mscreen)
    _values = data_im[idx, cam + '_Image'].reshape((len(idx), len(y3grid), len(x3grid))).sum(axis=(0, 1))
    fint = interpolate.interp1d(_points, _values, **kws)
    pws.append(fint(wgrid))
    XXP.append([x, xp])
pws = np.array(pws)
XXP = np.array(XXP)

In [ ]:
f_xxpw = np.zeros((len(xgrid), len(xpgrid), len(wgrid)))
_new_points = utils.get_grid_coords(xgrid, xpgrid)
for k in trange(len(wgrid)):
    _points = XXP
    _values = pws[:, k]
    _new_values = interpolate.griddata(_points, _values, _new_points, method='linear', fill_value=False)
    f_xxpw[:, :, k] = _new_values.reshape((len(xgrid), len(xpgrid)))

In [ ]:
_xgrid = xgrid - np.average(xgrid, weights=utils.project(f_xxpw, axis=0))
_xpgrid = xpgrid - np.average(xpgrid, weights=utils.project(f_xxpw, axis=1))
_wgrid = wgrid - np.average(wgrid, weights=utils.project(f_xxpw, axis=2))
# mplt.interactive_proj1d(f_xxpw, coords=[_xgrid, _xpgrid, _wgrid], dims=['x', 'xp', 'w'])

fig, ax = pplt.subplots()
ax.plot(_wgrid, utils.project(f_xxpw, axis=2))